### Imports

In [231]:
# import public packages
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys


from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction

### Add Directories / Load Files

In [ ]:
#reload(find_folders)
onedrive = find_folders.get_onedrive_path()
ftg_path = find_folders.get_onedrive_path("FTG")
print(ftg_path)

In [237]:
percept_ID = 'sub029'
fname = 'sync_sub-029_ses-EphysMedOn01_task-RampUpThres125_acq-StimOnR2b_run-01_ieeg.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        ftg_path,
        'data',
        'raw_data',
        'raw_mats',
        percept_ID,
        fname
    ),
    info = None
)

KeyError: 'data'

#### Artefact Rejection & FIF Files

In [235]:
raw.ch_names

['LFP_Stn_L_13',
 'LFP_Stn_R_13',
 'LFP_Stn_L_13_PEAK66Hz_THR20-30_AVG3000ms',
 'LFP_Stn_R_13_PEAK73Hz_THR20-30_AVG3000ms',
 'STIM_L_125Hz_60us',
 'STIM_R_125Hz_60us']

In [234]:
#Interactive plot for artefact rejection
%matplotlib qt 

fig = raw.plot(n_channels = 9, highpass = 5, lowpass = 100, 
    filtorder = 5, duration = 20)


Setting up band-pass filter from 5 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 5.00, 100.00 Hz: -6.02, -6.02 dB



Channels marked as bad:
none


In [ ]:
#First time making them
interactive_annot = raw.annotations
raw.annotations.save('Sub029_Bilateral_artefactsAnnotations.csv', 
    overwrite = True)

In [ ]:
#Importing them later
interactive_annot = pd.read_csv(
    os.path.join(
        ftg_path,
        'data',
        'raw_data',
        'clean_fifs',
        'sub029',
        'Sub029_RampUpThres_artefactsAnnotations.csv'
    )
)

#fix timing in imported annotations
reload(fix_annot_onsets)
new_onsets = fix_annot_onsets.fix_annot_onsets(interactive_annot)
print(new_onsets)

In [ ]:
my_annot = mne.Annotations(onset=new_onsets,  # in seconds
                           duration=interactive_annot.duration,  # in seconds, too
                           description=interactive_annot.description,
                           orig_time=raw.info['meas_date'])

reload(mat2fif)
%matplotlib qt
new_raw = mat2fif.mat2fif(raw,my_annot)

In [ ]:
#Plot TF plot with artefact rejected data
%matplotlib qt
reload(dat_preproc)
x = new_raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = new_raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

### Plot Power Spectra in epochs

In [ ]:
%matplotlib inline
stim_onsets = [1, 30, 110, 240]
labels = ['Ipsi_Stim','NoStim','Contra_Stim','Bilateral']
dur = 20

#for l in range(0,4):
    #plt.plot(np.mean(Sxx[1,:,stim_onsets[l]:stim_onsets[l]+dur],1), label = labels[l])
    #plt.xlim(40, 90)
    #plt.ylim(0,0.5)
tt, Pxx = scipy.signal.welch(Sxx[1,1:20], fs = 250, nperseg = 250, noverlap = 0.25)
plt.plot(np.arange(1,127), np.mean(Pxx[0],1), label = labels[l])

plt.legend(labels)

In [ ]:
gfp

In [ ]:
reload(dat_preproc)
x = new_raw.get_data(picks = 1) #getting only RSTN channel
base = x[0, 10000:17500]




x1 = x[0, 72500:80000] #taking 30 seconds of recording

dat_subh = dat_preproc.low_highpass_filter(x1, 60, 65) #filtering for 60-65Hz
dat_ngam = dat_preproc.low_highpass_filter(x1, 80, 90) #filtering for 65-90Hz
dat_bet = dat_preproc.low_highpass_filter(x1, 20, 35) #filtering for 13-35Hz

datall = [dat_bet, dat_subh, dat_ngam] 
labels = ['High Beta [20-35Hz]', 'Subharmonic [60-65Hz]','FTG [80-90Hz]']

In [ ]:
amplitude_envelope.shape[0]

In [ ]:
reload(baseline_correction)

In [ ]:
for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    times = amplitude_envelope.shape[0] * 1e3
    gfp = np.sum(amplitude_envelope ** 2, axis=0)
    gfp_bs = baseline_correction.baseline_corr(gfp, times, baseline = base)



    idx1 = idx + 1
    plt.subplot(3,1,idx1)
    plt.plot(np.arange(1,7501), gfp, label = labels[idx]) 
    plt.axvline(3750, color = 'b', ls='--', lw=2)
    plt.legend(loc = 'upper right')
    plt.xticks(np.arange(0,8000,1000), np.arange(0,32,4))
    plt.xlabel('Time [sec]')
    plt.ylabel('GPF')

In [ ]:
# let's explore some frequency bands
iter_freqs = [
    ('Beta', 13, 35),
    ('SubGamma', 60, 65),
    ('NarrGamma', 65, 90)
]

In [ ]:
new_raw.ch_names

In [ ]:
raw = new_raw
events = mne.find_events(raw, stim_channel='STIM_R_125Hz_60us')
print(events)

In [ ]:
# set epoching parameters
event_id, tmin, tmax = 1, -1., 3.
baseline = None


frequency_map = list()

for band, fmin, fmax in iter_freqs:
    # (re)load the data to save memory

    # bandpass filter
    raw.filter(fmin, fmax, n_jobs=None,  # use more jobs to speed up.
               l_trans_bandwidth=1,  # make sure filter params are the same
               h_trans_bandwidth=1, picks = 'LFP_Stn_R_13')  # in each band and skip "auto" option.

    # epoch
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, baseline=baseline,
                        picks= 'LFP_Stn_R_13',
                        preload=True)
    # remove evoked response

    # get analytic signal (envelope)
    epochs.apply_hilbert(envelope=True, picks = 'LFP_Stn_R_13')
    frequency_map.append(((band, fmin, fmax), epochs.average(picks = 'LFP_Stn_R_13')))
    del epochs
del raw

In [ ]:
np.sum(average.data, axis = 0).shape

In [ ]:
from mne.stats import bootstrap_confidence_interval

# Helper function for plotting spread
def stat_fun(x):
    """Return sum of squares."""
    return np.sum(x ** 2, axis=0)


# Plot
fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True, sharey=True)
colors = plt.colormaps['winter_r'](np.linspace(0, 1, 3))
for ((freq_name, fmin, fmax), average), color, ax in zip(
        frequency_map, colors, axes.ravel()[::-1]):
    times = average.times * 1e3
    gfp1 = np.sum(average.data ** 2, axis=0)
    gfp = mne.baseline.rescale(gfp1, times, baseline=(None, 0))
    ax.plot(times, gfp, label=freq_name, color=color, linewidth=2.5)
    ax.axhline(0, linestyle='--', color='grey', linewidth=2)
    ci_low, ci_up = bootstrap_confidence_interval(average.data, random_state=0,
                                                  stat_fun=stat_fun)
    ci_low = rescale(ci_low, average.times, baseline=(None, 0))
    ci_up = rescale(ci_up, average.times, baseline=(None, 0))
    ax.fill_between(times, gfp + ci_up, gfp - ci_low, color=color, alpha=0.3)
    ax.grid(True)
    ax.set_ylabel('GFP')
    ax.annotate('%s (%d-%dHz)' % (freq_name, fmin, fmax),
                xy=(0.95, 0.8),
                horizontalalignment='right',
                xycoords='axes fraction')
    ax.set_xlim(-1000, 3000)
    ax.set_ylim(-50,50)

axes.ravel()[-1].set_xlabel('Time [ms]')